# Code

### Imports

In [1]:
import inseq
import pandas as pd
import warnings
import byt5_model
import torch

warnings.filterwarnings("ignore")

### Functions

In [2]:
class InseqAttributer:
    """Create an object that loads a given model through Inseq with a given attribution method"""
    def __init__(self, model="./models/test/", attribution_method="input_x_gradient") -> None:
        self.model = inseq.load_model(model, attribution_method)
    
    def attribute(self, inp:str, out:str=None):
        """Use the Inseq model to generate feature attributions using a given input or input & output"""
        if inp and out:
            inp_out = (inp, out)
        else:
            inp_out = (inp,)

        inseq_out = self.model.attribute(*inp_out,
                                         attribute_target=True,
                                         step_scores=["probability"])
        inseq_out.show()

In [9]:
def data_sampler(data: str, model, samples: int=10, sampling: bool=True, spaces: bool=True, device: str='auto'):
    """Using the data from a given file, take a given amount of random samples 
    and a given model to generate the output of the model. 
    
    ---------
    Arguments
    ---------
    data: str
        a string containing the data for German or Turkish
    model 
        a Hugginface model
    samples: int
        a given number of samples to be generated
    sampling: bool
        Choose to use sampling or not
    spaces: bool
        Choose your input lemma feature set combination should use spaces for generation
    device: str
        Set the device that pytorch should use - cpu, gpu, or auto (gpu if possible)

    -------
    Returns
    -------
    Pandas DataFrame

    """
    if device == 'auto':
        device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
    elif device == 'gpu' or device == 'cuda':
        device = torch.device("cuda")
    elif device == 'cpu':
        device = torch.device("cpu")
    else:
        print('Set device to auto, cpu, or gpu/cuda.')
        return

    test = pd.read_csv(
        data,
        sep='\t',
        names=('lemma', 'labels', 'features')
    )

    if spaces:
        test['inputs'] = test['lemma'] + ' ' + test['features']
    else:
        test['inputs'] = test['lemma'] + test['features']

    test_gen_comparison = byt5_model.comparer(test, byt5_model.generate(model, test, device))
    test = pd.concat([test, test_gen_comparison], axis=1).rename(columns={'labels': 'plural'})

    correct = []
    for idx, row in test.iterrows():
        if row['Expected'] == row['Predicted']:
            correct.append('correct')
        else:
            correct.append('incorrect')

    test = test.assign(correct=correct)

    if sampling:
        return pd.concat([test[test.correct == 'incorrect'].sample(n=samples//2, random_state=1234),
                          test[test.correct == 'correct'].sample(n=samples//2, random_state=1234)])
    else:
        return pd.concat([test[test.correct == 'incorrect'],
                          test[test.correct == 'correct']])

In [4]:
def sample_runner(samples: pd.DataFrame, inseq_model, expected=False):
    """Run all samples from a given samples dataframe with a given inseq model.
    
    ---------
    Arguments
    ---------
    samples: DataFrame
        A DataFrame containing the samples for attribution analysis
    inseq_model
        The inseq model that has been prepared beforehand using InseqAttributer
    expected: bool
        Choose whether to use the values the model expects, try the correct gold label output

    """

    count = 0
    for idx, sample in samples.iterrows():
        count += 1
        if count > 1:
            print('\n')
        print('='*80)

        inp = sample['lemma'] + ' ' + sample['features']
        if expected:
            print('Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)')
            output_type = 'gold'
            out = sample['Expected']
        else:
            print('Testing Predicted (pred) outcome (what did the model take from the input during generation)')
            output_type = 'pred'
            out = sample['Predicted']

        print('Sample: {}\nIndex: {}\nThis prediction is {}\nInput -> output ({}): \'{}\' -> \'{}\''.format(
             count, idx, sample['correct'], output_type, inp, out
        ))
        if output_type == 'pred':
            print(f"The expected or gold output would be '{sample['Expected']}'")

        print('='*80)

        inseq_model.attribute(inp,
                              out)

# Testing

## German

### Setup

In [5]:
model_ger_finetuned = "./models/finetuned_ger"
inseq_ger_finetuned = InseqAttributer(model_ger_finetuned, "input_x_gradient")

In [6]:
model_ger_fromscratch = "./models/saved_model_scratch_ger_new"
inseq_ger_fromscratch = InseqAttributer(model_ger_fromscratch, "input_x_gradient")

In [7]:
model_ger2_finetuned = "./models/finetuned_ger_2"
inseq_ger2_finetuned = InseqAttributer(model_ger2_finetuned, "input_x_gradient")

In [10]:
test = data_sampler('./data/deu.gold', model_ger_finetuned, sampling=False)

        Expected     Predicted
0         Orgien        Orgien
1         Sieger        Sieger
2         Klötze        Klotze
3          Kalke         Kalke
4       Skelette      Skelette
5        Flocken       Flocken
6       Schwänze      Schwänze
7       Schwämme      Schwämme
8          Kegel         Kegel
9         Geckos        Geckon
10       Trauben       Trauben
11        Pfeile        Pfeile
12       Stollen       Stollen
13  Scheinwerfer  Scheinwerfer
14     Köchinnen     Köchinnen
15    Vorsprünge    Vorsprünge
16        Salons        Salons
17      Umstände      Umstände
18     Schlingel     Schlingel
19  Markierungen  Markierungen


In [12]:
test[test.correct == 'correct'].sample(10)

,lemma,plural,features,inputs,Expected,Predicted,correct
502,Gewässer,Gewässer,N;NOM;NEUT;PL,Gewässer N;NOM;NEUT;PL,Gewässer,Gewässer,correct
398,Blättchen,Blättchen,N;NOM;NEUT;PL,Blättchen N;NOM;NEUT;PL,Blättchen,Blättchen,correct
130,Winter,Winter,N;NOM;MASC;PL,Winter N;NOM;MASC;PL,Winter,Winter,correct
160,Lämmchen,Lämmchen,N;NOM;NEUT;PL,Lämmchen N;NOM;NEUT;PL,Lämmchen,Lämmchen,correct
483,Kehle,Kehlen,N;NOM;FEM;PL,Kehle N;NOM;FEM;PL,Kehlen,Kehlen,correct
485,Wolle,Wollen,N;NOM;FEM;PL,Wolle N;NOM;FEM;PL,Wollen,Wollen,correct
27,Gewürz,Gewürze,N;NOM;NEUT;PL,Gewürz N;NOM;NEUT;PL,Gewürze,Gewürze,correct
403,Prinzip,Prinzipe,N;NOM;NEUT;PL,Prinzip N;NOM;NEUT;PL,Prinzipe,Prinzipe,correct
462,Schmierpapier,Schmierpapiere,N;NOM;NEUT;PL,Schmierpapier N;NOM;NEUT;PL,Schmierpapiere,Schmierpapiere,correct
432,Klumpen,Klumpen,N;NOM;MASC;PL,Klumpen N;NOM;MASC;PL,Klumpen,Klumpen,correct


### German Finetuned V2

The sample_runner function splits the given amount of samples in `two` (default is 10) so that `~50% of the samples are samples of correct predictions`, and `~50% of the samples are samples of incorrect predictions`.

First the incorrect predictions are shown, then the correct predictions are shown.

#### Model's predictions

In [13]:
sample_runner(data_sampler('./data/deu.gold', model_ger2_finetuned, samples=20), inseq_ger2_finetuned, expected=False)

        Expected     Predicted
0         Orgien        Orgien
1         Sieger        Sieger
2         Klötze        Klotze
3          Kalke         Kalke
4       Skelette      Skelette
5        Flocken       Flocken
6       Schwänze      Schwänze
7       Schwämme      Schwämme
8          Kegel         Kegel
9         Geckos        Geckon
10       Trauben       Trauben
11        Pfeile        Pfeile
12       Stollen       Stollen
13  Scheinwerfer  Scheinwerfer
14     Köchinnen     Köchinnen
15    Vorsprünge    Vorsprünge
16        Salons        Salons
17      Umstände      Umstände
18     Schlingel     Schlingel
19  Markierungen  Markierungen
Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 1
Index: 228
This prediction is incorrect
Input -> output (pred): 'Abschleppwagen N;NOM;MASC;PL' -> 'Abschleppwägen'
The expected or gold output would be 'Abschleppwagen'


Attributing with input_x_gradient...: 100%|██████████| 17/17 [00:07<00:00,  2.25it/s]


,A,b,s,c,h,l,e,p,p,w,Ã,¤,g,e,n,</s>
A,0.041,0.11,0.094,0.054,0.018,0.11,0.1,0.099,0.056,0.078,0.049,0.042,0.096,0.042,0.032,0.051
b,0.057,0.255,0.089,0.055,0.021,0.108,0.035,0.035,0.03,0.028,0.021,0.031,0.088,0.11,0.052,0.117
s,0.035,0.034,0.065,0.036,0.026,0.041,0.066,0.042,0.035,0.021,0.053,0.032,0.061,0.023,0.056,0.019
c,0.01,0.013,0.058,0.101,0.056,0.014,0.018,0.017,0.015,0.019,0.01,0.028,0.018,0.011,0.017,0.029
h,0.012,0.021,0.013,0.018,0.034,0.014,0.018,0.026,0.018,0.022,0.019,0.016,0.017,0.014,0.014,0.02
l,0.009,0.01,0.033,0.05,0.03,0.044,0.021,0.048,0.02,0.029,0.017,0.021,0.027,0.017,0.015,0.006
e,0.009,0.008,0.011,0.023,0.026,0.016,0.049,0.019,0.02,0.021,0.032,0.014,0.008,0.017,0.012,0.003
p,0.013,0.016,0.031,0.025,0.012,0.032,0.034,0.029,0.052,0.023,0.014,0.024,0.018,0.013,0.014,0.011
p,0.026,0.008,0.013,0.025,0.013,0.026,0.03,0.023,0.144,0.045,0.019,0.054,0.016,0.014,0.05,0.008
w,0.012,0.008,0.012,0.024,0.011,0.011,0.015,0.033,0.043,0.051,0.029,0.047,0.018,0.026,0.03,0.014




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 2
Index: 464
This prediction is incorrect
Input -> output (pred): 'Streit N;NOM;MASC;PL' -> 'Streiten'
The expected or gold output would be 'Streite'


Attributing with input_x_gradient...: 100%|██████████| 10/10 [00:04<00:00,  2.17it/s]


...

Attributing with input_x_gradient...: 100%|██████████| 11/11 [00:04<00:00,  2.22it/s]


,S,c,h,l,Ã,¤,m,m,e,</s>
S,0.092,0.086,0.076,0.039,0.056,0.122,0.067,0.086,0.038,0.078
c,0.122,0.127,0.036,0.086,0.031,0.081,0.126,0.08,0.023,0.031
h,0.033,0.038,0.044,0.056,0.032,0.036,0.023,0.036,0.05,0.052
l,0.039,0.025,0.048,0.057,0.048,0.036,0.041,0.016,0.027,0.047
a,0.008,0.032,0.031,0.04,0.105,0.04,0.024,0.028,0.052,0.011
m,0.019,0.044,0.055,0.04,0.034,0.036,0.042,0.057,0.058,0.017
m,0.028,0.019,0.046,0.046,0.026,0.024,0.059,0.049,0.031,0.037
,0.075,0.027,0.023,0.027,0.053,0.015,0.035,0.051,0.027,0.021
N,0.032,0.02,0.024,0.031,0.02,0.012,0.025,0.028,0.014,0.014
;,0.04,0.028,0.031,0.052,0.038,0.024,0.028,0.032,0.046,0.091




Testing Predicted (pred) outcome (what did the model take from the input during generation)
Sample: 20
Index: 282
This prediction is correct
Input -> output (pred): 'Metzger N;NOM;MASC;PL' -> 'Metzger'
The expected or gold output would be 'Metzger'


Attributing with input_x_gradient...: 100%|██████████| 9/9 [00:03<00:00,  2.16it/s]


,M,e,t,z,g,e,r,</s>
M,0.082,0.086,0.091,0.045,0.061,0.04,0.046,0.045
e,0.086,0.064,0.065,0.045,0.085,0.025,0.015,0.032
t,0.053,0.097,0.123,0.085,0.096,0.096,0.088,0.014
z,0.03,0.018,0.027,0.095,0.036,0.061,0.038,0.032
g,0.033,0.053,0.047,0.046,0.033,0.033,0.026,0.023
e,0.011,0.013,0.019,0.019,0.019,0.047,0.023,0.014
r,0.044,0.104,0.08,0.069,0.053,0.034,0.068,0.03
,0.031,0.024,0.019,0.021,0.02,0.018,0.038,0.026
N,0.022,0.017,0.017,0.013,0.021,0.017,0.014,0.03
;,0.034,0.029,0.031,0.025,0.029,0.033,0.033,0.094


#### What the model thinks of the Gold predictions (only relevant for the incorrect items)

In [14]:
sample_runner(data_sampler('./data/deu.gold', model_ger2_finetuned, samples=20), inseq_ger2_finetuned, expected=True)

        Expected     Predicted
0         Orgien        Orgien
1         Sieger        Sieger
2         Klötze        Klotze
3          Kalke         Kalke
4       Skelette      Skelette
5        Flocken       Flocken
6       Schwänze      Schwänze
7       Schwämme      Schwämme
8          Kegel         Kegel
9         Geckos        Geckon
10       Trauben       Trauben
11        Pfeile        Pfeile
12       Stollen       Stollen
13  Scheinwerfer  Scheinwerfer
14     Köchinnen     Köchinnen
15    Vorsprünge    Vorsprünge
16        Salons        Salons
17      Umstände      Umstände
18     Schlingel     Schlingel
19  Markierungen  Markierungen
Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 1
Index: 228
This prediction is incorrect
Input -> output (gold): 'Abschleppwagen N;NOM;MASC;PL' -> 'Abschleppwagen'


Attributing with input_x_gradient...: 100%|██████████| 16/16 [00:05<00:00,  2.81it/s]


,A,b,s,c,h,l,e,p,p,w,a,g,e,n,</s>
A,0.041,0.11,0.094,0.054,0.018,0.11,0.1,0.099,0.056,0.078,0.049,0.113,0.052,0.066,0.054
b,0.057,0.255,0.089,0.055,0.021,0.108,0.035,0.035,0.03,0.028,0.021,0.096,0.15,0.015,0.091
s,0.035,0.034,0.065,0.036,0.026,0.041,0.066,0.042,0.035,0.021,0.053,0.053,0.026,0.059,0.011
c,0.01,0.013,0.058,0.101,0.056,0.014,0.018,0.017,0.015,0.019,0.01,0.015,0.01,0.029,0.02
h,0.012,0.021,0.013,0.018,0.034,0.014,0.018,0.026,0.018,0.022,0.019,0.014,0.017,0.012,0.021
l,0.009,0.01,0.033,0.05,0.03,0.044,0.021,0.048,0.02,0.029,0.017,0.028,0.018,0.023,0.007
e,0.009,0.008,0.011,0.023,0.026,0.016,0.049,0.019,0.02,0.021,0.032,0.008,0.014,0.012,0.003
p,0.013,0.016,0.031,0.025,0.012,0.032,0.034,0.029,0.052,0.023,0.014,0.022,0.013,0.019,0.011
p,0.026,0.008,0.013,0.025,0.013,0.026,0.03,0.023,0.144,0.045,0.019,0.029,0.01,0.077,0.014
w,0.012,0.008,0.012,0.024,0.011,0.011,0.015,0.033,0.043,0.051,0.029,0.018,0.025,0.024,0.019




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 2
Index: 464
This prediction is incorrect
Input -> output (gold): 'Streit N;NOM;MASC;PL' -> 'Streite'


Attributing with input_x_gradient...: 100%|██████████| 9/9 [00:02<00:00,  3.98it/s]


...

Attributing with input_x_gradient...: 100%|██████████| 11/11 [00:04<00:00,  2.21it/s]


,S,c,h,l,Ã,¤,m,m,e,</s>
S,0.092,0.086,0.076,0.039,0.056,0.122,0.067,0.086,0.038,0.078
c,0.122,0.127,0.036,0.086,0.031,0.081,0.126,0.08,0.023,0.031
h,0.033,0.038,0.044,0.056,0.032,0.036,0.023,0.036,0.05,0.052
l,0.039,0.025,0.048,0.057,0.048,0.036,0.041,0.016,0.027,0.047
a,0.008,0.032,0.031,0.04,0.105,0.04,0.024,0.028,0.052,0.011
m,0.019,0.044,0.055,0.04,0.034,0.036,0.042,0.057,0.058,0.017
m,0.028,0.019,0.046,0.046,0.026,0.024,0.059,0.049,0.031,0.037
,0.075,0.027,0.023,0.027,0.053,0.015,0.035,0.051,0.027,0.021
N,0.032,0.02,0.024,0.031,0.02,0.012,0.025,0.028,0.014,0.014
;,0.04,0.028,0.031,0.052,0.038,0.024,0.028,0.032,0.046,0.091




Testing Expected (gold) outcome (what would the model look at to get to the expected outcome)
Sample: 20
Index: 282
This prediction is correct
Input -> output (gold): 'Metzger N;NOM;MASC;PL' -> 'Metzger'


Attributing with input_x_gradient...: 100%|██████████| 9/9 [00:03<00:00,  2.19it/s]


,M,e,t,z,g,e,r,</s>
M,0.082,0.086,0.091,0.045,0.061,0.04,0.046,0.045
e,0.086,0.064,0.065,0.045,0.085,0.025,0.015,0.032
t,0.053,0.097,0.123,0.085,0.096,0.096,0.088,0.014
z,0.03,0.018,0.027,0.095,0.036,0.061,0.038,0.032
g,0.033,0.053,0.047,0.046,0.033,0.033,0.026,0.023
e,0.011,0.013,0.019,0.019,0.019,0.047,0.023,0.014
r,0.044,0.104,0.08,0.069,0.053,0.034,0.068,0.03
,0.031,0.024,0.019,0.021,0.02,0.018,0.038,0.026
N,0.022,0.017,0.017,0.013,0.021,0.017,0.014,0.03
;,0.034,0.029,0.031,0.025,0.029,0.033,0.033,0.094
